In [1]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message,ibConnection
import ib.opt
import ib.ext.Contract
from ib.ext.TickType import TickType as tt
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [117]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 5
        self.rcdId = 1000

        self.con = ibConnection(**kwargs)
        
        self.con.registerAll(self.watcher)
        self.contracts = {}
        self.prices = pd.DataFrame(columns = ['id','bidPrice','askPrice'])
        self.lastTime = pd.DataFrame(columns = ['id','bidPrice','askPrice'])


        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler,
            ib.opt.message.tickPrice: self.tickPriceHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()
        #time.sleep(1)
        #self.con.reqMarketDataType( 3 )
        

    
    def addContract(self,contract):
        n= self.prices.shape[0]
        id = 100+n
        self.contracts[id] = contract
        self.prices.loc[n] = pd.Series({'id':id,'bidPrice':0,'askPrice':0})
        self.lastTime.loc[n] = pd.Series({'id':id,'bidPrice':0,'askPrice':0})
    
    def tickPriceHandler(self, msg):
        #print (msg,msg.tickerId,msg.field,msg.price)
        if msg.field in [1,2]:
            print (msg.tickerId,msg.field,msg.price)
            if msg.field == 1:
                field = 'bidPrice'
                
                if 'frontOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.frontOptStruct.index:
                        strike = self.frontOptStruct.loc[msg.tickerId].m_strike
                        right = self.frontOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.frontOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.frontOptPrices.loc[strike,'CALL BID'] = msg.price
                if 'secondOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.secondOptStruct.index:
                        strike = self.secondOptStruct.loc[msg.tickerId].m_strike
                        right = self.secondOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.secondOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.secondOptPrices.loc[strike,'CALL BID'] = msg.price
                if 'esOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.esOptStruct.index:
                        strike = self.esOptStruct.loc[msg.tickerId].m_strike
                        right = self.esOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.esOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.esOptPrices.loc[strike,'CALL BID'] = msg.price
                
            if msg.field == 2:
                field = 'askPrice'  
                
                if 'frontOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.frontOptStruct.index:
                        strike = self.frontOptStruct.loc[msg.tickerId].m_strike
                        right = self.frontOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.frontOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.frontOptPrices.loc[strike,'CALL ASK'] = msg.price
                if 'secondOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.secondOptStruct.index:
                        strike = self.secondOptStruct.loc[msg.tickerId].m_strike
                        right = self.secondOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.secondOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.secondOptPrices.loc[strike,'CALL ASK'] = msg.price
                if 'esOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.esOptStruct.index:
                        strike = self.esOptStruct.loc[msg.tickerId].m_strike
                        right = self.esOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.esOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.esOptPrices.loc[strike,'CALL ASK'] = msg.price
                        
            self.prices.loc[self.prices.id==msg.tickerId,field] = msg.price

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))
            
    def get_market_data(self):
        print ("Calling Market Data")
        for key in self.contracts.keys():          
            self.con.reqMktData(key, self.contracts[key], None, True)
            time.sleep(0.01)

    def get_contract_details(self, contract):
        self.rcdId=self.rcdId+1
        self.con.reqContractDetails(self.rcdId, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            try:
                if df.shape[0]==0:
                    df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                    df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
                else:
                    df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            except:
                pass

        return df, err, mid, msg

In [190]:
def vixFutContract(localSymbol=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)

    return contract

def spFutContract(localSymbol=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    #contract.m_tradingClass = 'SP'
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)
    
    return contract

def esFutContract(localSymbol=None):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)
    
    return contract

def vxOptContract(exp=None, localSymbol=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)
    contract.m_expiry = exp
    contract.m_tradingClass='VIX'
 
    return contract

def spOptContract(exp=None, localSymbol=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    contract.m_expiry = exp
    #contract.m_tradingClass='SPX'
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)
 
    return contract

def esOptContract(exp=None, localSymbol=None):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    contract.m_expiry = exp
    contract.m_tradingClass="ES"
    if localSymbol:
        contract.m_localSymbol = str(localSymbol)
 
    return contract

In [192]:
#ibm.con.disconnect()
ibm = IbManager("127.0.0.1", port=7497, clientId=999)

Server Version: 76
TWS Time at connection:20181219 10:56:47 EST
('----------', <ib.opt.message.ManagedAccounts object at 0x0D253770>)
('----------', <ib.opt.message.NextValidId object at 0x0DDE2F50>)
('----------', <ib.opt.message.Error object at 0x12A81080>)
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:hfarm>
('----------', <ib.opt.message.Error object at 0x12A81080>)
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
('----------', <ib.opt.message.Error object at 0x12A81D28>)
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
('----------', <ib.opt.message.Error object at 0x12A81D28>)
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture.nj>
('----------', <ib.opt.message.Error object at 0x12A81D28>)
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:jfarm>
('----------', <ib.opt.message.Error object at 0x12A81300>)
<error id=-1, errorCode=2104

In [193]:
#read config
cfg = pd.read_json('config.json')
cfg

,name,optionsExp,symbol
0,second,20190521,VXG9
1,front,20190115,VXF9
2,sp,20190117,SPH9


In [194]:
ibm.prices

,id,bidPrice,askPrice


In [195]:
ibm.addContract(vixFutContract(cfg[cfg.name=='front'].symbol.values[0]))
ibm.addContract(vixFutContract(cfg[cfg.name=='second'].symbol.values[0]))
ibm.addContract(spFutContract(localSymbol=cfg[cfg.name=='sp'].symbol.values[0]))

In [196]:
#create dataFrame for VIX options
optCtrct = vxOptContract(str(cfg[cfg.name=='front'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.frontOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.frontOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = vxOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [197]:
#create dataFrame for VIX options
optCtrct = vxOptContract(str(cfg[cfg.name=='second'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.secondOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.secondOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = vxOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [198]:
#create dataFrame for ES options
optCtrct = spOptContract(str(cfg[cfg.name=='sp'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.esOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.esOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = spOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [208]:
ibm.get_market_data()

Calling Market Data
('----------', <ib.opt.message.MarketDataType object at 0x0D49B1D0>)
(100, 1, 21.4)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
(100, 2, 21.45)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickString object at 0x0E192800>)
('----------', <ib.opt.message.TickGeneric object at 0x0E192800>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0D381390>)
('----------', <ib.opt.message.MarketDataType object at 0x0E059E10>)
(101, 1, 21.2)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
(101, 2, 21.25)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickSize object at 0x0E192800>)
('----------', <ib.opt.message.TickString object at 0x0E192800>)
('--------

('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13F97D70>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13F97D70>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13F97D70>)
('----------', <ib.opt.message.TickString object at 0x119A5B70>)
('----------', <ib.opt.message.TickString object at 0x119A5B70>)
('----------', <ib.opt.message.TickString object at 0x119A5B70>)
('----------', <ib.opt.message.TickGeneric object at 0x119A5B70>)
('----------', <ib.opt.message.MarketDataType object at 0x0DDF2E10>)
(131, 1, 3.4)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
(131, 2, 3.6)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickS

('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8530>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8BB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8E30>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8BB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8E30>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8BB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8E30>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8270>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8BB0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0E059E10>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8270>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8E30>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FB8BB0>)
('----------', <ib

(155, 2, 11.7)
('----------', <ib.opt.message.TickSize object at 0x04C343C8>)
('----------', <ib.opt.message.TickString object at 0x04C343C8>)
('----------', <ib.opt.message.TickString object at 0x04C343C8>)
(152, 1, 8.3)
('----------', <ib.opt.message.TickSize object at 0x125CCC60>)
(152, 2, 8.5)
('----------', <ib.opt.message.TickSize object at 0x125CCC60>)
('----------', <ib.opt.message.TickString object at 0x125CCC60>)
('----------', <ib.opt.message.TickString object at 0x125CCC60>)
(154, 1, 9.2)
('----------', <ib.opt.message.TickSize object at 0x125CCC60>)
(154, 2, 9.4)
('----------', <ib.opt.message.TickSize object at 0x125CCC60>)
('----------', <ib.opt.message.TickString object at 0x125CCC60>)
('----------', <ib.opt.message.TickString object at 0x125CCC60>)
('----------', <ib.opt.message.MarketDataType object at 0x0D3D4F10>)
(158, 1, 0.4)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
(158, 2, 0.45)
('----------', <ib.opt.message.TickSize object at 0x0E287BC0>)


('----------', <ib.opt.message.MarketDataType object at 0x0D3230B0>)
(175, 1, 0.05)
('----------', <ib.opt.message.TickSize object at 0x0CEE48C8>)
(175, 2, 0.1)
('----------', <ib.opt.message.TickSize object at 0x0CEE48C8>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0D38CCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0D38CCB0>)
('----------', <ib.opt.message.TickString object at 0x0CEE48C8>)
('----------', <ib.opt.message.TickString object at 0x0CEE48C8>)
('----------', <ib.opt.message.TickGeneric object at 0x0CEE48C8>)
('----------', <ib.opt.message.MarketDataType object at 0x0DDF2E10>)
('----------', <ib.opt.message.MarketDataType object at 0x0D3D4F10>)
(178, 1, 0.05)
('----------', <ib.opt.message.TickSize object at 0x0E1B5378>)
(178, 2, 0.1)
('----------', <ib.opt.message.TickSize object at 0x0E1B5378>)
('----------', <ib.opt.message.TickString object at 0x0E1B5378>)
('----------', <ib.opt.message.TickString object at 0x0E1B5378>)
('---

...

('----------', <ib.opt.message.TickSize object at 0x125CCC60>)
(814, 2, 609.2)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
(808, 1, 586.0)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
(808, 2, 589.3)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
(816, 1, 616.1)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
(816, 2, 619.4)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
('----------', <ib.opt.message.TickString object at 0x134F5F80>)
(840, 1, 1124.9)
('----------', <ib.opt.message.TickSize object at 0x134F5F80>)
(840, 2, 1128.1)
('----------', <ib.opt.message.TickSize object at 0x134

('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------

('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA52B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA52B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA52B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA52B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------

('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0E059E10>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA5030>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0D49B1D0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA5030>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0DDBFF50>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0DDF2E10>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA5030>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0D441F30>)
('----------', <ib.opt.message.TickOptionC

('----------', <ib.opt.message.TickOptionComputation object at 0x0CE7DCB0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0DDC7CD0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA5C70>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0DDBFF50>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0DDF2E10>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA5E70>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA54B0>)
('----------', <ib.opt.message.TickSnapshotEnd object at 0x0D3D4F10>)
('----------', <ib.opt.message.TickOptionComputation object at 0x13FA50B0>)
('----------', <ib.opt.message.TickO

In [217]:
def tickHandler(msg):
    print (msg)
ibm.con.register(tickHandler, ib.opt.message.tickPrice)

True

In [1]:
ibm.frontOptStruct

NameError: name 'ibm' is not defined